# Data Analysis of Data Analysts

Project 1

In [50]:
# Dependencies and Setup
import os
import csv
import pandas as pd
from pathlib import Path
from matplotlib import pyplot as plt
import matplotlib.pyplot as plt
from scipy.stats import sem
from scipy.stats import linregress
import scipy.stats as st
import numpy as np
import requests
import json
import matplotlib.pyplot as plt
import cartopy.crs as ccrs



In [51]:
from config import acct_id
from config import api_key 

#Creation of an empty Data Frame for storage of values per State & City
city_state = {
    'City': ["Atlanta", "Austin", "San Francisco", "New York City", "San Diego", "Boston", "Portland",
            "Seattle", "Los Angeles", "Chicago", "Denver", "Dallas"],
    'State': ["GA", "TX", "CA", "NY", "CA", "MA", "OR", "WA", "CA", "IL", "CO", "TX"]}
    
dataframe_1 = pd.DataFrame(city_state)
dataframe_1[['# of Analysts - City', 'Median Salary - City','Median Salary - State']] = ""

dataframe_1

,City,State,# of Analysts - City,Median Salary - City,Median Salary - State
0,Atlanta,GA,,,
1,Austin,TX,,,
2,San Francisco,CA,,,
3,New York City,NY,,,
4,San Diego,CA,,,
5,Boston,MA,,,
6,Portland,OR,,,
7,Seattle,WA,,,
8,Los Angeles,CA,,,
9,Chicago,IL,,,


In [52]:
api_url = f'''https://api.careeronestop.org/v1/occupation/{acct_id}/Data%20Scientists/0?training=false
&interest=false&videos=false&tasks=false&dwas=false&wages=true
&alternateOnetTitles=false&projectedEmployment=false&ooh=false
&stateLMILinks=false&relatedOnetTitles=false&skills=false&knowledge=false
&ability=false&trainingPrograms=false'''

headers = {"Authorization": f"Bearer {api_key}"}
response = requests.get(api_url, headers=headers).json()
#response

print(json.dumps(response, indent=4, sort_keys=True))

{
    "AutoCorrection": "",
    "DidYouMean": "",
    "OccupationDetail": [
        {
            "AlternateTitles": null,
            "BrightOutlook": "Bright",
            "BrightOutlookCategory": "Rapid Growth",
            "COSVideoURL": "https://www.careeronestop.org/Videos/careeronestop-videos.aspx?videocode=15205100&op=y",
            "Green": "No",
            "Location": "US",
            "OnetCode": "15-2051.00",
            "OnetDescription": "Develop and implement a set of techniques or analytics applications to transform raw data into meaningful information using data-oriented programming languages and visualization software. Apply data mining, data modeling, natural language processing, and machine learning to extract and analyze information from large structured and unstructured datasets. Visualize, interpret, and report data findings. May create dynamic data reports.",
            "OnetTitle": "Data Scientists",
            "RelatedOnetTitles": null,
            "SocInf

In [53]:
# Use the city API request but alter the city and state based on the dataframe
# This allows us to search national, state, and city data all from the same call

base_url = city_url = f"https://api.careeronestop.org/v1/occupation/{acct_id}/Data%20Scientists/"
params = "?training=false&interest=true&videos=false&tasks=false&dwas=false&wages=true&alternateOnetTitles=false&projectedEmployment=true&ooh=false&stateLMILinks=false&relatedOnetTitles=false&skills=false&knowledge=false&ability=false&trainingPrograms=false"
headers = {"Authorization": f"Bearer {api_key}"}

In [54]:
response["OccupationDetail"][0]["Wages"]["NationalWagesList"][0]["Median"]

'103500'

In [ ]:
for index, row in dataframe_1.iterrows():
    # Get the city and state
    city = dataframe_1.loc[index, "City"]
    state = dataframe_1.loc[index, "State"]
    # Add the current City type to the parameters
    api_url =  f"{base_url}{city}%2C%20{state}{params}"
    
    response_1 = requests.get(api_url, headers = headers).json()
    results = response_1['OccupationDetail'][0]['Wages']['BLSAreaWagesList']
    results_state = response_1['OccupationDetail'][0]['Wages']['StateWagesList']
#     results_2 = response["OccupationDetail"][0]['Projections']['Projections'][0]['ProjectedEmployment']

    try:
        #Projected Jobs
        projected_jobs = response_1["OccupationDetail"][0]['Projections']['Projections'][0]['EstimatedEmployment']
        dataframe_1.loc[index, '# of Analysts - City'] = projected_jobs
        
#       help from tutor Median Salart for city
        median_salary = [x['Median'] for x in results if x['RateType']=='Annual' ][0] 
        dataframe_1.loc[index, 'Median Salary - City'] = median_salary

        #Median Salary for State
        median_salary_state = [x['Median'] for x in results_state if x['RateType']=='Annual' ][0] 
        dataframe_1.loc[index, 'Median Salary - State'] = median_salary_state 
        
    except (KeyError, IndexError):
        dataframe_1.loc[index, 'Median Salary - City'] = "Didnt find anything"
        dataframe_1.loc[index, '# of Analysts - City'] = "Didnt find anything"
        dataframe_1.loc[index, 'Median Salaty - State'] = "Didnt find anything"
        

dataframe_1

In [63]:
#Formatting into Dollars and Alhpabetize
formatted_df = dataframe_1.copy()
formatted_df = formatted_df.sort_values(by='City', ascending=True)
formatted_df['Median Salary - City'] = formatted_df['Median Salary - City'].astype(float).map("${:,.2f}".format)
formatted_df['Median Salary - State'] = formatted_df['Median Salary - State'].astype(float).map("${:,.2f}".format)

formatted_df

,City,State,# of Analysts - City,Median Salary - City,Median Salary - State
0,Atlanta,GA,"2,250","$104,060.00","$100,400.00"
1,Austin,TX,"4,110","$105,350.00","$99,350.00"
5,Boston,MA,"2,880","$106,670.00","$106,070.00"
9,Chicago,IL,"3,370","$104,370.00","$103,730.00"
11,Dallas,TX,"4,110","$103,330.00","$99,350.00"
10,Denver,CO,740,"$103,800.00","$103,200.00"
8,Los Angeles,CA,"18,400","$103,430.00","$126,810.00"
3,New York City,NY,"3,500","$130,070.00","$124,030.00"
6,Portland,OR,760,"$108,130.00","$102,530.00"
4,San Diego,CA,"18,400","$123,570.00","$126,810.00"


In [60]:
city_data =[]

city_response = requests.get(api_url, headers=headers).json()

city_median = city_response["OccupationDetail"][0]["Wages"]["BLSAreaWagesList"][1]["Median"]
city_median
# for index, row in df.iterrows():
#     # Extract necessary data from the row
#     param_value = row['BLSAreaWagesList']
    
#     # Define parameters for the API call
#     params = {'Median': param_value}

#city_response["OccupationDetail"][0]["Wages"]["BLSAreaWagesList"][0]["Median"]
#city_response

'103330'